# Parameters

In [ ]:
### roboflow
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="eiYUOfvvafXOToxBN05P")
project = rf.workspace("dorna").project("cardamom-good-bad")
version = project.version(28)
dataset = version.download("darknet")

### parameters ###
trained_model_path = "weight/yolov4_tiny_cardamom_good_bad_max_v5" # make the directory on the google drive if it doesn't exists
trained_model_type = "last" # use last or final
percentage_test = 5
pretrained_model = "yolov4-tiny.conv.29"
cfg = "yolov4-tiny-custom"

# get the package
!pip install albumentations

# imports
from google.colab import drive
import glob, os
import shutil
import albumentations as A
import cv2
import random

# Augmentation Dictionary
augmentation = {
    "run_augmentations": 0,  # Set to 1 to apply augmentations, 0 to skip
    # The p value in all of these augmentations represents the probability of that augmentation being applied.
    # "gauss_noise": {"var_limit": (10.0, 50.0), "p": 0},  # The var_limit is the range of variance for adding Gaussian noise to the image.
    # "gaussian_blur": {"blur_limit": (3, 7), "p": 0},  # The blur_limit is the range of kernel sizes for applying Gaussian blur to the image.
    # "random_brightness_contrast": {"brightness_limit": 0.2, "contrast_limit": 0.2, "p": 0},  # The brightness limit defines the change in brightness. Ex: 0.2 means +/-20% change. The contrast limit defines the change in contrast. Ex: 0.2 means +/-20% change.
    # "random_gamma": {"gamma_limit": (50, 130), "p": 0},  # The gamma limit defines the range of gamma values for gamma correction. Ex: (50, 130) means the gamma value will be chosen between 50 and 130.
    # "iso_noise": {"color_shift": (0, 0), "intensity": (0, 0), "p": 0},  # The color shift defines the range of color shift for ISO noise. The intensity defines the range of intensity for ISO noise.
    # "to_gray": {"p": 0},  # The p value represents the probability of converting the image to grayscale.
    # "hue_saturation_value": {"hue_shift_limit": 20, "sat_shift_limit": 0, "val_shift_limit": 40, "p": 0.2},  # The hue shift limit defines the maximum change in hue. The sat shift limit defines the maximum change in saturation. The val shift limit defines the maximum change in value (brightness).
    # "random_scale": {"scale_limit": (0.5, 3), "p": 0.5},  # The scale limit defines the range for scaling the image. Ex: (0.6, 2) means the scale factor will be chosen between 0.6 (60%) and 2 (200%). The interpolation method to use. Ex: 1 corresponds to linear interpolation.
}

# Imports

In [ ]:
# change cfg
def update_cfg_file(cfg_file_path, line_changes):
    try:
        # Read the file
        with open(cfg_file_path, 'r') as file:
            lines = file.readlines()

        # Modify specific lines
        for line_number, new_value in line_changes.items():
            if 0 < line_number <= len(lines):
                lines[line_number - 1] = new_value  # Adjusting for 0-based index
            else:
                print(f"Line {line_number} is out of range. Skipping.")

        # Write the changes back to the file
        with open(cfg_file_path, 'w') as file:
            file.writelines(lines)

        print(f"Updated specific lines in {cfg_file_path}.")

    except FileNotFoundError:
        print(f"File {cfg_file_path} not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


# helper function
def data_prep(percentage_test, current_dir="/content/darknet/data"):

  file_train = open(current_dir+'/train.txt', 'w')
  file_test = open(current_dir+'/test.txt', 'w')

  counter = 1
  index_test = round(100 / percentage_test)
  for pathAndFilename in glob.iglob(os.path.join(current_dir+"/obj/", "*.jpg")):
      title, ext = os.path.splitext(os.path.basename(pathAndFilename))

      if counter == index_test:
          counter = 1
          file_test.write(current_dir + "/obj/" + title + '.jpg' + "\n")
      else:
          file_train.write(current_dir + "/obj/" + title + '.jpg' + "\n")
          counter = counter + 1


# create .obj data
def create_obj_data(classes, file_path="/content/darknet/data/obj.data"):
    # Define the content with a placeholder for classes
    content = f"""classes = {classes}
train  = /content/darknet/data/train.txt
valid  = /content/darknet/data/test.txt
names = /content/darknet/data/obj.names
backup = /content/darknet/backup
"""
    # Ensure the directory exists
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    # Write the content to the file
    with open(file_path, 'w') as file:
        file.write(content)

# find classes
def find_classes(folder_path):
    # Search for .labels files in the specified folder
    labels_files = [f for f in os.listdir(folder_path) if f.endswith('.labels')]

    # If no .labels file is found, return an empty list
    if not labels_files:
        return []

    # Assuming we want the first found .labels file
    labels_file_path = os.path.join(folder_path, labels_files[0])

    # Read the lines of the file into a list
    with open(labels_file_path, 'r') as file:
        lines = file.readlines()

    return [line.strip() for line in lines]  # Remove any extra newline characters

def count_images(image_folder):
    return len([f for f in os.listdir(image_folder) if f.endswith('.jpg') or f.endswith('.png')])

# dataset
dataset_path = dataset.location

# connect to drive
drive.mount('/content/drive')


# clone the darknet repo
!git clone https://github.com/AlexeyAB/darknet

# make
%cd /content/darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile
!make

# create cfg
%cd data/
!find -maxdepth 1 -type f -exec rm -rf {} \;
%cd ..

# find num_classes, max_batches, filters, steps
classes = find_classes(dataset_path+"/train")
num_classes = len(classes)

# backup backup files
!mkdir /content/darknet/backup

# move all the train data to data
!mkdir /content/darknet/data/obj
!mv "$dataset_path"/train/* /content/darknet/data/obj/

# Create the obj.data file
create_obj_data(num_classes)

#change the directory to match that of the names file in your drive.
!mv /content/darknet/data/obj/_darknet.labels /content/darknet/data/obj.names

# pretrained weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/"$pretrained_model"


#Augmentations
bbox_params = A.BboxParams(format='yolo', label_fields=['class_labels'])

def get_transforms(augmentation):
    transforms = []
    if augmentation["gaussian_blur"]["p"] > 0:
        transforms.append(A.GaussianBlur(blur_limit=augmentation["gaussian_blur"]["blur_limit"], p=augmentation["gaussian_blur"]["p"]))
    if augmentation["random_brightness_contrast"]["p"] > 0:
        transforms.append(A.RandomBrightnessContrast(brightness_limit=augmentation["random_brightness_contrast"]["brightness_limit"], contrast_limit=augmentation["random_brightness_contrast"]["contrast_limit"], p=augmentation["random_brightness_contrast"]["p"]))
    if augmentation["random_gamma"]["p"] > 0:
        transforms.append(A.RandomGamma(gamma_limit=augmentation["random_gamma"]["gamma_limit"], p=augmentation["random_gamma"]["p"]))
    if augmentation["hue_saturation_value"]["p"] > 0:
        transforms.append(A.HueSaturationValue(hue_shift_limit=augmentation["hue_saturation_value"]["hue_shift_limit"], sat_shift_limit=augmentation["hue_saturation_value"]["sat_shift_limit"], val_shift_limit=augmentation["hue_saturation_value"]["val_shift_limit"], p=augmentation["hue_saturation_value"]["p"]))
    if augmentation["random_scale"]["p"] > 0:
        transforms.append(A.RandomScale(scale_limit=augmentation["random_scale"]["scale_limit"], p=augmentation["random_scale"]["p"]))
    #if augmentation["shift_scale_rotate"]["p"] > 0:
    #    transforms.append(A.ShiftScaleRotate(shift_limit=augmentation["shift_scale_rotate"]["shift_limit"], scale_limit=augmentation["shift_scale_rotate"]["scale_limit"], rotate_limit=augmentation["shift_scale_rotate"]["rotate_limit"], p=augmentation["shift_scale_rotate"]["p"]))
    #if augmentation["random_dynamic_resized_crop"]["p"] > 0:
    #    transforms.append(A.RandomResizedCrop(height=augmentation["random_dynamic_resized_crop"]["max_size"], width=augmentation["random_dynamic_resized_crop"]["max_size"], scale=augmentation["random_dynamic_resized_crop"]["scale"], p=augmentation["random_dynamic_resized_crop"]["p"]))
    #if augmentation["pad_if_needed"]["p"] > 0:
    #     transforms.append(A.PadIfNeeded(min_height=augmentation["pad_if_needed"]["min_height"], min_width=augmentation["pad_if_needed"]["min_width"], border_mode=augmentation["pad_if_needed"]["padding_mode"], p=augmentation["pad_if_needed"]["p"]))

    return A.Compose(transforms, bbox_params=bbox_params) if transforms else None

def augment_image(image, bboxes, class_labels, num_augmentations, transform):
    """Apply augmentations multiple times to an image and its bounding boxes."""
    augmented_images = []
    augmented_bboxes_list = []
    augmented_labels_list = []

    for _ in range(num_augmentations):
        augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        augmented_images.append(augmented['image'])
        augmented_bboxes_list.append(augmented['bboxes'])
        augmented_labels_list.append(augmented['class_labels'])

    return augmented_images, augmented_bboxes_list, augmented_labels_list

def process_directory(image_dir, annotations_dir, output_dir, num_augmentations=5):
    """Process images and annotations in a directory."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')]

    # Get transformation based on the augmentation dictionary
    transform = get_transforms(augmentation) if augmentation["run_augmentations"] == 1 else None

    for image_file in image_files:
        # Load image
        image_path = os.path.join(image_dir, image_file)
        image = cv2.imread(image_path)

        # Load bounding boxes
        annotation_file = image_file.replace('.jpg', '.txt').replace('.png', '.txt')
        annotations_path = os.path.join(annotations_dir, annotation_file)
        bboxes = []
        class_labels = []
        with open(annotations_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    class_labels.append(int(parts[0]))
                    x_center, y_center, width, height = map(float, parts[1:])
                    bboxes.append([x_center, y_center, width, height])

        if transform:
            # Augment image and bounding boxes multiple times
            augmented_images, augmented_bboxes_list, augmented_labels_list = augment_image(image, bboxes, class_labels, num_augmentations, transform)

            # Save augmented images and bounding boxes
            for i, (augmented_image, augmented_bboxes, augmented_labels) in enumerate(zip(augmented_images, augmented_bboxes_list, augmented_labels_list)):
                # Save augmented image
                augmented_image_path = os.path.join(output_dir, f"{os.path.splitext(image_file)[0]}_aug_{i}.jpg")
                cv2.imwrite(augmented_image_path, augmented_image)

                # Save augmented bounding boxes with their correct class labels
                augmented_annotations_path = os.path.join(output_dir, f"{os.path.splitext(image_file)[0]}_aug_{i}.txt")
                with open(augmented_annotations_path, 'w') as f:
                    for label, bbox in zip(augmented_labels, augmented_bboxes):
                        f.write(f"{label} {' '.join(map(str, bbox))}\n")
        else:
            # If no augmentation, just copy the original image and annotations
            output_image_path = os.path.join(output_dir, image_file)
            cv2.imwrite(output_image_path, image)

            output_annotation_path = os.path.join(output_dir, annotation_file)
            with open(annotations_path, 'r') as f:
                content = f.read()
            with open(output_annotation_path, 'w') as f:
                f.write(content)

# Path to the directories
image_dir = '/content/darknet/data/obj'
annotations_dir = '/content/darknet/data/obj'
output_dir = '/content/darknet/data/obj'

# Process the directory
process_directory(image_dir, annotations_dir, output_dir, num_augmentations=5)

print("Augmentation complete.")
# create train and test
data_prep(percentage_test)

# Directory with images
image_folder = "/content/darknet/data/obj"
# Count the number of images
num_images = count_images(image_folder)
# Calculate max_batches
max_batches = max(6000, num_classes * 2000, num_images)
# Calculate steps
steps = f"{int(max_batches * 0.8)},{int(max_batches * 0.9)}"
# lines to be changed
line_changes = {
    6: 'batch=64\n',
    7: 'subdivisions=16\n',
    8: 'width=416\n',
    9: 'height=416\n',
    20: f'max_batches={max_batches}\n',  # Adjust based on `num_classes`, `num_images`
    22: f'steps={steps}\n',  # Adjust based on `max_batches`
    220: f'classes={num_classes}\n',  # Adjust based on `num_classes`
    269: f'classes={num_classes}\n',  # Adjust based on `num_classes`
    212: f'filters={(num_classes + 5) * 3}\n',  # Adjust based on `num_classes`
    263: f'filters={(num_classes + 5) * 3}\n'   # Adjust based on `num_classes`
}

update_cfg_file("/content/darknet/cfg/"+cfg+".cfg", line_changes)

# Train the model

In [ ]:
#This is the training block, it will take some time.
!./darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/"$cfg".cfg /content/darknet/"$pretrained_model"

# **Evaluation**

In [ ]:
# model path
model_path = "/content/darknet/backup/"+cfg+"_"+trained_model_type+".weights"
cfg_path = "/content/darknet/cfg/"+cfg+".cfg"

#Set your custom cfg to evaluation mode.
!sed -i 's/batch=64/batch=1/' "$cfg_path"
!sed -i 's/subdivisions=16/subdivisions=1/' "$cfg_path"

In [ ]:
#Use can use this line to check the mAP for all the diffrent weights to see which gives the best results (change the directory to match yours).
!./darknet detector map /content/darknet/data/obj.data "$cfg_path" "$model_path" -points 0

In [ ]:
#Run on Image (change the directories to match yours).
# Find the result at /content/darknet/predictions.jpg
test_image_path = "/content/5053170388753821383.jpg"
!./darknet detector test /content/darknet/data/obj.data "$cfg_path" "$model_path" "$test_image_path" -thresh 0.5

# Darknet -> ncnn



In [ ]:
# Clone the NCNN repository
%cd /content
!git clone https://github.com/Tencent/ncnn.git
%cd ncnn

# Create a build directory and navigate to it
!rm -rf build
!mkdir build
%cd build

# Configure the build
!cmake ..
!make -j4

# darknet to ncnn
%cd /content/ncnn/build/tools/darknet
!./darknet2ncnn "$cfg_path" "$model_path" /content/model.param /content/model.bin 1

# optimize
%cd ..
!./ncnnoptimize /content/model.param /content/model.bin /content/model_opt.param /content/model_opt.bin 0


# Save data

In [ ]:
# Copy the optimized model to Google Drive
shutil.copy("/content/model_opt.bin", "/content/drive/My Drive/"+trained_model_path+".bin")
shutil.copy("/content/model_opt.param", "/content/drive/My Drive/"+trained_model_path+".param")

# Copy the file to Google Drive
shutil.copy(model_path, "/content/drive/My Drive/"+trained_model_path+".weights")
shutil.copy(cfg_path, "/content/drive/My Drive/"+trained_model_path+".cfg")